In [22]:
from util import parse_db, Document, read_markup_tsv
import random
from torch.utils.data import Dataset, DataLoader

records = parse_db("data_/0525_parsed.db")


In [23]:
import sentence_transformers
from sentence_transformers import CrossEncoder, InputExample
from torch.utils.data import DataLoader
 
model = CrossEncoder('bert-base-multilingual-uncased', num_labels= 1)

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model 

In [25]:
label2int = {"BAD": 0, "OK": 1}

random.seed(0)

def separate_samples(markup, train_percent=0.85):

    count_samples = len(markup)
    shuffled_ids = list(range(count_samples))
    random.shuffle(shuffled_ids)
    train_len = round(count_samples * train_percent)

    train_markup, test_markup = [], []

    for i, id in enumerate(shuffled_ids):
      if i < train_len:
        train_markup.append(markup[id])
      else:
        test_markup.append(markup[id])        
         
    return train_markup, test_markup


def get_data(markup, records):
    input_samples, other_samples, qualities = [], [], []

    for mrkp in markup:
        first_url, second_url, quality = mrkp.values()

        input_samples.append(url2record[first_url]['title'] + ' ' + url2record[first_url]['text'])
        other_samples.append(url2record[second_url]['title'] + ' ' + url2record[second_url]['text'])
     
        qualities.append(quality)
    return input_samples, other_samples, qualities

url2record = dict()

for i, record in enumerate(records):
    url2record[record["url"]] = record


markup = read_markup_tsv("data_/ru_clustering_0525_urls.tsv")
train_markup, test_markup = separate_samples(markup)
train_inputs, train_others, train_qualities  = get_data(train_markup, records)
test_inputs, test_others, test_qualities  = get_data(test_markup, records)

In [26]:
train_examples = [InputExample(texts=[inp, oth], label = label2int[qual]) for inp, oth, qual in zip(train_inputs, train_others, train_qualities)]
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)

test_examples = [InputExample(texts=[inp, oth], label = label2int[qual]) for inp, oth, qual in zip(test_inputs, test_others, test_qualities)]
test_dataloader = DataLoader(test_examples, shuffle=True, batch_size=16)


In [27]:
evaluator = sentence_transformers.evaluation.LabelAccuracyEvaluator(test_dataloader)

In [28]:
model.fit(train_dataloader=train_dataloader,
          evaluator = evaluator,
          epochs=5)

AttributeError: 'CrossEncoder' object has no attribute 'eval'